# Titanic: Machine Learning from Disaster

One of the most popular and known events is certainly the sinking of the magnificent Titanic on April 15, 1912 - the ship which was considered to be "unsinkable" at the time. Unfortunately, the Titanic sank after colliding with an iceberg which led to a death of 1502 out of 2224 passengers and crew. 

Surviving the collision was not purely the result of luck or chance - there were some factors which led to more a likely surviving of some passengers. Some of those factors include (and are certainly not limited to) gender and age, as we know from the popular movie depicting the disaster.

The goal of this project is to explore dataset containing information on Titanic passengers and to build the model which would predict whether someone survived or not as accurate as possible. Furthermore, the model should give us an information on which factors were actually most important when the crew was determining who is going to be saved by lifeboats.

# Importing Libraries and Modules 

In [1]:
from warnings import simplefilter
simplefilter(action = 'ignore', category = FutureWarning)
import numpy as np
import pandas as pd 
import plotly.express as px 
import plotly.graph_objects as go 
import plotly.io as pio 
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef as phi
import researchpy

In [2]:
# Setting default settings for graphs
pio.renderers.default = 'browser'
pio.templates.default = 'simple_white'

# 1. Exploring Data

In [3]:
titanic = pd.read_csv('train.csv')
titanic_copy = titanic.copy()
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The whole dataset was already split into trainining and testing data by Kaggle. Therefore, only training data were imported and there won't be a need for splitting data any further.

By examining the columns and types, we can see that there are 12 features:

1. PassengerId - ID of passenger (no NAs)
2. Survived - Whether passenger survived or not (no NAs)
3. Pclass - Ticket class: first, second, or third (no NAs)
4. Name - Full passenger's name (no NAs)
5. Sex - Whether passenger was male or female (no NAs)
6. Age - Age in years (NAs present)
7. SibSp - The number of siblings/spouses aboard the Titanic (no NAs)
8. Parch - The number of parents/children aboard the Titanic (no NAs)
9. Ticket - Ticket number (no NAs)
10. Fare - Passenger fare (no NAs)
11. Cabin - Cabin number (NAs present)
12. Embarked - Port of Embarkation: Cherbourg, Queenstown, or Southampton (NAs present)

Furthermore, we can see that only a few features have missing values, and those are *Age*, *Cabin number*, and *Port of Embarkation*. 

This section named **Exploring data** will be separated into subsections:

1. Excluding Outliers
2. Handling Missing Values
3. Excluding Irrelevant Features 
4. Visualizing Distributions

## 1.1. Excluding Outliers

Detecting and excluding outliers is a really important part of every project, since they can distort the final predictions. There are many ways to conclude whether an observation is an outlier value or not. Fidell & Tabachnik (2003) suggest using a Z value of |3.29| when we do not have a strong theoretical basis for any other approach. However, that approach often fails to detect outliers, since outliers impact standard deviation and Z values can be inflated. 

Therefore, we opt for an more robust approach of inter-quartile range, also known as Tukey's method. There are a total of four different numerical features that could be examined for outliers - *Age, SibSp, Parch,* and *Fare*. However, we are going to examine only Age and Fare here for one simple reason - SibSp and Parch can be combined into a new feature which could indicate how big the family is. Furthermore, we can observe the rank of these features and see that *SibSp* and *Parch* have restricted ranks, which mean that their variance probably is small and therefore not very useful for final predictions. If we exclude outliers for those features, we will remain with a little information that could be extracted from these features. However, combining those two features could help in overcoming that problem or could lead to discretizyng the feature in several different categories.

In [5]:
print('Rank for Siblings and/or Spuses is', titanic['SibSp'].max() - titanic['SibSp'].min())
print('Rank for Parents and/or Children is', titanic['Parch'].max() - titanic['Parch'].min())

Rank for Siblings and/or Spuses is 8
Rank for Parents and/or Children is 6


In [6]:
def outliers(df, column, new_column):
    """Detects outliers and appends as a new column to the dataset.

    Args:
        df (pandas DataFrame): Dataframe with relevant feature
        column (pandas Series): Feature in focus
        new_column (str): The name of the new feature
    
    Returns:
        pandas Series
    """
    q1 = np.percentile(df[column], 25)
    q3 = np.percentile(df[column], 75)
    iqr = q3 - q1
    
    # Multiplying iqr by 3 to get lower and upper outer boundaries
    outer = iqr * 3
    lower = q1 - outer
    upper = q3 + outer
    # Identifying outliers
    for index, value in titanic[column].items():
        if value < lower:
            titanic.loc[index, new_column] = 1
        elif value > upper:
            titanic.loc[index, new_column] = 1
        else:
            titanic.loc[index, new_column] = 0
    return titanic

In [7]:
# Saving outliers
titanic = outliers(titanic, 'Age', 'Age_outliers')
titanic = outliers(titanic, 'Fare', 'Fare_outliers')

In [8]:
print('A total number of outlying values for Age is', titanic['Age_outliers'].sum())
print('A total number of outlying values for Fare is', titanic['Fare_outliers'].sum())

A total number of outlying values for Age is 0.0
A total number of outlying values for Fare is 53.0


In [9]:
# Excluding outliers for Fare feature
titanic = titanic.loc[titanic['Fare_outliers'] == 0, :]
titanic = titanic.drop(columns = ['Age_outliers', 'Fare_outliers'])
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
# Number of training instances after excluding outliers
titanic.iloc[:, 1].size

838

## 1.2. Handling Missing Data

Before we correct missing values in training set, we should check the test set as well in order to see whether the pattern of missing values is similar. In other words, the problem could occur if we develop a model based on features which have a lot of missing values in testing set.

In [11]:
titanic_test = pd.read_csv('test.csv')
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


By observing the output of the previous line, we can conclude that the missing values pattern is similar - *Age*, *Cabin*, and *Fare* have missing values (with *Fare* having only one missing value!). Therefore, we can continue with handling missing data from the training set!

Let us deal firstly with the feature *Embarked* since it has only two missing values. Since the number of missing values is small and the feature is categorial, the best solution is to input the mode value for missing values.

In [12]:
titanic['Embarked'].describe()

count     836
unique      3
top         S
freq      620
Name: Embarked, dtype: object

In [13]:
titanic.loc[titanic['Embarked'].isnull(), 'Embarked'] = 'S'
titanic['Embarked'].isnull().sum()

0

Now, let us explore *Age* feature and conclude what is the best way to handle the missing data.

In [14]:
titanic['Age'].describe()

count    666.000000
mean      29.536411
std       14.522137
min        0.420000
25%       20.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

We can see that the average age was around 30, while the median (50th percentile) is 28, which means passengers were particularly young. However, there is a minimum value of 0.42, which is a bit confusing since the age is coded as years, not as months. However, maybe that was a baby and they actually got the information on how many months the baby was old. Let us explore whether that was the case.

In [15]:
sum(titanic['Age'] < 1)

6

There was a total of 6 passengers that were not even one year old. A quick Google search reveiled that there actually were some babies on board (https://www.theguardian.com/world/2009/jun/01/last-titanic-survivor-dies). Therefore, we are not going to exclude these observations for now. Nevertheless, let us explore the correlation between age and other variables! Maybe we can input missing features based on predictions rather on the mean value. For those purposes, we are going to use Linear Regression while obtaining R squared measure and F statistic as well.

In [16]:
# Encoding sex as numerical feature
titanic.loc[titanic['Sex'] == 'male', 'Sex'] = 1
titanic.loc[titanic['Sex'] == 'female', 'Sex'] = 2
titanic['Sex'] = titanic['Sex'].astype(int)
titanic['Sex'].unique()

# Saving only features useful for the actual predictions
titanic_age = titanic[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
titanic_age = pd.get_dummies(titanic_age, columns = ['Pclass', 'Embarked'], drop_first = True)
titanic_age.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,0,1,22.0,1,0,7.2500,0,1,0,1
1,1,2,38.0,1,0,71.2833,0,0,0,0
2,1,2,26.0,0,0,7.9250,0,1,0,1
3,1,2,35.0,1,0,53.1000,0,0,0,1
4,0,1,35.0,0,0,8.0500,0,1,0,1


In [17]:
import statsmodels.api as sm

# Dropping PassengerId feature since it is not informative in any way
X = titanic_age.dropna()[['Survived', 'Sex', 'SibSp', 'Parch', 'Fare', 'Pclass_2', 'Pclass_3', 'Embarked_Q', 'Embarked_S']]
y = titanic_age.dropna()["Age"]

model = sm.OLS(y, X).fit()

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    Age   R-squared (uncentered):                   0.807
Model:                            OLS   Adj. R-squared (uncentered):              0.804
Method:                 Least Squares   F-statistic:                              305.2
Date:                Thu, 11 Jun 2020   Prob (F-statistic):                   5.40e-228
Time:                        21:09:35   Log-Likelihood:                         -2724.0
No. Observations:                 666   AIC:                                      5466.
Df Residuals:                     657   BIC:                                      5507.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Survived      -7.5873      1.447     -5.244      0.000     -10.428      -4.747
Sex            9.1963      1.313      7.002      0.000       6.617      11.775
SibSp         -6.3117      0.696     -9.067      0.000      -7.679      -4.945
Parch         -3.2966      0.757     -4.354      0.000      -4.783      -1.810
Fare           0.3971      0.034     11.722      0.000       0.331       0.464
Pclass_2       4.9258      1.859      2.650      0.008       1.276       8.576
Pclass_3       4.4605      1.780      2.506      0.012       0.965       7.956
Embarked_Q    11.7208      3.136      3.737      0.000       5.562      17.879
Embarked_S    12.0155      1.491      8.058      0.000       9.088      14.943
==============================================================================
Omnibus:                       15.403   Durbin-Watson:                   1.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               15.976
Skew:                           0.347   Prob(JB):                     0.000340
Kurtosis:                       3.306   Cond. No.                         189.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can see that Adjusted R-Squared is pretty high (0.804), which means that we can explain around 80% of Age variance. Hence, we can use the predictions of this model to input missing values, since those will be considerably better than using just the mean values. Even though the model was not cross-validated, the predictions are going to be better than simply inputting mean values.

In [18]:
# Predicting missing values for age
missing_age = titanic_age[titanic_age['Age'].isnull()]['Age']
predicting_age = titanic_age[titanic_age['Age'].isnull()][['Survived', 'Sex', 'SibSp', 'Parch', 'Fare', 'Pclass_2', 'Pclass_3', 'Embarked_Q', 'Embarked_S']]
missing_age = model.predict(predicting_age)
missing_age.head()

5     28.735943
17    23.712008
19    18.134434
26    16.525483
28    30.114961
dtype: float64

In [19]:
# Assigning predictions of missing age values
titanic.loc[titanic['Age'].isnull(), 'Age'] = missing_age
titanic['Age'].isnull().sum()

0

Since we have finished handling missing values for *Age* feature, we can move onto *Cabin* feature. 

*Cabin* has only 161 non-missing values, which means that roughly 75% of the values are actuall missing, which is not a promising start. Before excluding it, let us explore it a bit.

In [20]:
titanic['Cabin'].describe()

count     161
unique    120
top        G6
freq        4
Name: Cabin, dtype: object

A total of 120 unique values are present! That means that even if we keep this feature as it is, we will have a total of 1119 predicting features in our model after dummy coding! 

Furthermore, the total number of staterooms (suites, cabins, and dormitories) were 840 (source: https://titanicfacts.net/life-on-the-titanic/). That means that even if we approximate missing values, we would still operate within the known values which could systematically distort the final predictions. In other words, we don't know if any of the reamining cabins are actually the same ones we already have in our dataset.

After a short internet research, we can even see how the present values were obtained. The only authoritative source of cabin data was the incomplete first class passenger 
list recovered with the body of stewart Herbert Cave (source: https://www.encyclopedia-titanica.org/cabins.html), which means the information on cabins is systematically distorted. 

Furthermore, we cannot know for sure whether missing values indicate some other cabins or the fact that the passenger did not have a cabin. Therefore, it is best to exclude it for now, since predictions could be biased and unreliable.

In [21]:
titanic = titanic.drop(columns = 'Cabin')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",2,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",2,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",2,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,S


## 1.3. Excluding Irrelevant Features

Before continuing with further exploration of data, we should make a checkpoint of the current dataset and the feature relevance. 
After excluding *Class* feature, we are left with 11 other features - *PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare,* and *Embarked*. Let us quickly assess whether some features are useful at all.

- *PassengerId* is in no way useful, since it just contains values ranging from 1 to 891. Therefore, we can **exclude PassengerId** as well.

In [22]:
titanic = titanic.drop(columns = ['PassengerId'])

- *Ticket* feature is pretty similar to *PassengerId* - it contains values that are not meaningful and entirely depend on the system for generating ticket numbers. Therefore, we can safely **exclude Ticket** as well!

In [23]:
titanic = titanic.drop(columns = ['Ticket'])

- *Name* feature could potentially be useful since it contains information on whether someone was male or female or had some important title. However, we already have information on gender. As for the important titles, we should firstly check whether their frequency would indicate that we are dealing with outliers - in which case, the whole variable should be excluded as well.

In [24]:
titanic['Name']

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 838, dtype: object

In [25]:
titanic_title = []
for index, value in titanic['Name'].items():
    title = value.split(",")[1].split(".")[0].strip()
    titanic_title.append(title)
titanic["Title"] = pd.Series(titanic_title)
titanic["Title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', nan], dtype=object)

In [26]:
rare_titles = ['Don', 'Rev', 'Dr', 'Major', 'Lady', 'Sir', 'Col', 'Capt', 'the Countess', 'Jonkheer']
title_frequency = {'rare' : 0}
for index, value in titanic['Title'].items():
    if value in rare_titles:
        title_frequency['rare'] += 1
title_frequency['rare']

20

We can see that there are 20 people with rare titles (such as *Lord, Lady, Don, Dr*, etc.). Since we already have the information on gender and age (and can create more precise interaction feature), we can safely transform non-rare titles into label *"Not Rare"* and **exclude Name** feature.

In [27]:
for index, value in titanic['Title'].items():
    if value in rare_titles:
        titanic.loc[index, 'Title'] = 'Rare'
    else:
        titanic.loc[index, 'Title'] = 'Not Rare'

In [28]:
titanic = titanic.drop(columns = 'Name')

That leaves us with **9 final features** - *Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Titles,* and *Embarked*. In the end, we will change the numerical values for categorical features into text so we can more easily navigate the dataset. We can easily encode labels afterwards.

In [29]:
for index,value in titanic['Survived'].items():
    if value == 0:
        titanic.loc[index, 'Survived'] = "Didn't Survive"
    else:
        titanic.loc[index, 'Survived'] = 'Survived'

for index, value in titanic['Pclass'].items():
    if value == 1:
        titanic.loc[index, 'Pclass'] = 'First Class'
    elif value == 2:
        titanic.loc[index, 'Pclass'] = 'Second Class'
    else:
        titanic.loc[index, 'Pclass'] = 'Third Class'

for index, value in titanic['Embarked'].items():
    if value == 'S':
        titanic.loc[index, 'Embarked'] = 'Southampton'
    elif value == 'Q':
        titanic.loc[index, 'Embarked'] = 'Queenstown'
    else:
        titanic.loc[index, 'Embarked'] = 'Cherbourg'

for index, value in titanic['Sex'].items():
    if value == 1:
        titanic.loc[index, 'Sex'] = 'Male'
    else:
        titanic.loc[index, 'Sex'] = 'Female'

titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,Didn't Survive,Third Class,Male,22.0,1,0,7.2500,Southampton,Not Rare
1,Survived,First Class,Female,38.0,1,0,71.2833,Cherbourg,Not Rare
2,Survived,Third Class,Female,26.0,0,0,7.9250,Southampton,Not Rare
3,Survived,First Class,Female,35.0,1,0,53.1000,Southampton,Not Rare
4,Didn't Survive,Third Class,Male,35.0,0,0,8.0500,Southampton,Not Rare


In [30]:
# Descriptive statistics for numerical features
titanic.describe()

,Age,SibSp,Parch,Fare
count,838.000000,838.000000,838.000000,838.000000
mean,28.798459,0.514320,0.349642,22.411942
std,13.645432,1.117541,0.784067,20.827218
min,-3.799200,0.000000,0.000000,0.000000
25%,21.000000,0.000000,0.000000,7.895800
50%,28.447265,0.000000,0.000000,13.000000
75%,36.000000,1.000000,0.000000,27.720800
max,80.000000,8.000000,6.000000,93.500000


By observing descriptive statistics for numerical features in the table above, we can see that some observations have negative values for *Age* feature, which is not possible. That is probably due to linear regression model used to predict missing values. Therefore, we should exclude those observations now!

In [31]:
titanic = titanic[titanic['Age'] >= 0]

In [32]:
# Descriptive statistics for categorical features
titanic.describe(include = 'O')

,Survived,Pclass,Sex,Embarked,Title
count,834,834,834,834,834
unique,2,3,2,3,2
top,Didn't Survive,Third Class,Male,Southampton,Not Rare
freq,531,487,554,618,814


## 1.4. Visualizing Distributions

In [33]:
# Subplots with distributions
distributions = make_subplots(
    rows = 2, 
    cols = 5, 
    subplot_titles = ['Survived', 'Ticket Class', 'Sex', 'Port of Embarkation', 'Title', 'Age', 'Siblings and/or Spouses', 'Parents and/or Children', 'Fare']
)

cat_zip = list(
    zip(
        [1, 2, 3, 4, 5], ['Survived', 'Pclass', 'Sex', 'Embarked', 'Title']
        )
    )

for i in cat_zip:
    distributions.add_trace(
        go.Histogram(
            x = titanic[i[1]], 
            ), 
        row = 1, 
        col = i[0]
    )

num_zip = list(
    zip(
        [1, 2, 3, 4], ['Age', 'SibSp', 'Parch', 'Fare']
        )
    )

for i in num_zip:
    distributions.add_trace(
        go.Histogram(
            x = titanic[i[1]]
            ), 
        row = 2, 
        col = i[0]
    )

distributions.update_layout(
    showlegend = False, colorway = px.colors.qualitative.D3,
    title = 'Distribution of Features',
    title_x = 0.5,
    height = 650,
    width = 1100,
    plot_bgcolor = 'rgb(244, 244, 244)',
    paper_bgcolor = 'rgb(244, 244, 244)'
)

distributions.show(renderer = 'notebook_connected')

Let us qickly describe the graph.
As for the categorical features, we can see that distribution of levels is not equal for any of the present features, which was expected. Since outcome feature is not equally distributed, we will penalize developed model more when the error for predicting less frequent class occurs.

On the other hand, we can see that, out of all numerical features, only *Age* has nearly bell-shaped distribution. We will not perform a distribution transformation for features *Siblings/Spouses* and *Parents/Children* yet, since we are probably going to combine them into one feature and probably exclude a few outliers. Therefore, we'll perform transformation only for *Fare* feature now.

In [34]:
for index, value in titanic['Fare'].items():
    if value > 0:
        titanic.loc[index, 'Fare'] = np.sqrt(value)
    else:
        titanic.loc[index, 'Fare'] = 0

In [35]:
px.histogram(
    data_frame = titanic, 
    x = 'Fare',
    title = 'Fare Distribution'
).update_layout(
    title = 'Fare Distribution after Transformation', 
    title_x = 0.5,
    height = 400, 
    width = 500
).show(
    renderer = 'notebook_connected'
)

print(titanic['Age'].skew())

0.47010691249303427


# 2. Exploring Correlations and Feature Engineering

This section is separated into two parts.

**The first part** deals with exploring pivot tables and correlations, as well as visualizing data with regards to the outcome feature. The goal of this section is to obtain insights into factors which could potentially be useful for classification.

**The second part** deals with feature combinations. In essence, we are going to see whether combining different features (e.g. finding interactions, summing them up, etc.) will result in higher correlations with the outcome feature.

Before we proceed with exploring correlations, we should revert textual values of categorical features into numerical values and then dummy code them. Since we are going to use dummy coding in the final prediction, it is useful to have a sense of important categories through examining Phi coefficient of correlation between bivariate features!

In [36]:
for index,value in titanic['Survived'].items():
    if value == "Didn't Survive":
        titanic.loc[index, 'Survived'] = 0
    else:
        titanic.loc[index, 'Survived'] = 1

titanic['Survived'] = titanic['Survived'].astype('int16')

## 2.1. Pivot Tables and Correlations

### 2.1.1. Numerical Features

In [37]:
# Correlations between numerical features and survivorship
titanic.corr(
    # Using spearman's rho coefficient for obtaining more robust measure of correlation
    method = 'spearman'
)['Survived'].sort_values()

Age        -0.043183
SibSp       0.088890
Parch       0.139223
Fare        0.292331
Survived    1.000000
Name: Survived, dtype: float64

We can see the correlation is the highest when it comes to *Fare* feature. Furthermore, we can see that Age has surprisingly low correlation. However, maybe coming up with an interaction feature between Age and some other features will yield higher correlations. We leave that for the second part. Now, let us combine *Parch* and *SibSp* feature into a new one - *Family* feature. The new feature will indicate a number of family members.

In [38]:
titanic['Family'] = titanic['SibSp'] + titanic['Parch'] + 1
titanic.corr(
    method = 'spearman'
)['Survived'].sort_values()

Age        -0.043183
SibSp       0.088890
Parch       0.139223
Family      0.168373
Fare        0.292331
Survived    1.000000
Name: Survived, dtype: float64

Having a new feature "Family" leads to an increase in correlation. Therefore, we can exclude 'Parch' and 'SibSp' features since keeping them in dataset could lead to multicollinearity problems.

In [39]:
titanic = titanic.drop(columns = ['SibSp', 'Parch'])

Now, let us check the number of outlier values for our new feature!


In [40]:
titanic = outliers(titanic, 'Family', 'Family_outliers')
titanic['Family_outliers'].sum()

39.0

We can see that a total of 43 values are outliers. However, excluding that much values can lead to loosing some important information, especially information which comes from other features. Therefore, it is probably a smarter move to discretize this variable into several features. Before doing that, we should examine counts of different values to get a sense of the number of categories.

In [41]:
titanic['Family'].value_counts()

1     521
2     146
3      93
4      22
6      18
5      13
7      12
8       6
11      3
Name: Family, dtype: int64

Seems like we should combine this into three main categories - traveling alone, traveling with someone and traveling with more than one member of the family. That way, we can overcome excluding 43 observations.

In [42]:
for index, value in titanic['Family'].items():
    if value == 1:
        titanic.loc[index, 'Family'] = 'Alone'
    elif value == 2:
        titanic.loc[index, 'Family'] = 'Small_f'
    else:
        titanic.loc[index, 'Family'] = 'Big_f'
titanic = titanic.drop(columns = 'Family_outliers')
titanic.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,Family
0,0,Third Class,Male,22.0,2.692582,Southampton,Not Rare,Small_f
1,1,First Class,Female,38.0,8.442944,Cherbourg,Not Rare,Small_f
2,1,Third Class,Female,26.0,2.815138,Southampton,Not Rare,Alone
3,1,First Class,Female,35.0,7.286975,Southampton,Not Rare,Small_f
4,0,Third Class,Male,35.0,2.837252,Southampton,Not Rare,Alone


Since we are dealing with categorical feature for now, we are going to leave final approximation of correlation for the next part. Right now, we should visualize the correlations between numerical features and the outcome feature using heatmap.

In [43]:
corr_num = titanic[['Survived', 'Age', 'Fare']].corr(method = 'spearman').round(3).to_numpy()
corr_num_plot = ff.create_annotated_heatmap(
    corr_num,
    x = ['Survived', 'Age', 'Fare'],
    y = ['Survived', 'Age', 'Fare'],
    colorscale = 'Teal'
)

corr_num_plot.update_layout(
    title = 'Correlations between Outcome and Numerical Features',
    title_x = 0.5,
    width = 500,
    height = 400
)

corr_num_plot.show(renderer = 'notebook_connected')

### 2.1.2. Categorical Features

Categorical features should be considered separately, since measures such as Pearson's, Spearman's, or Kendall's coefficients of correlations cannot quantify the relationship between categorical features. Hence, some other measures should be used, such as *Cramer's V* and *Phi coefficient* (for binary data). 

In [44]:
for i in ['Pclass', 'Sex', 'Embarked', 'Title', 'Family']:
    print(
        'Feature:', 
        i, 
        '\n', 
        researchpy.crosstab(
            titanic['Survived'], titanic[i], test = 'chi-square')[1], 
        '\n'
        )

Feature: Pclass 
                 Chi-square test  results
0  Pearson Chi-square ( 2.0) =   77.1893
1                    p-value =    0.0000
2                 Cramer's V =    0.3042 

Feature: Sex 
                 Chi-square test   results
0  Pearson Chi-square ( 1.0) =   229.0651
1                    p-value =     0.0000
2               Cramer's phi =     0.5241 

Feature: Embarked 
                 Chi-square test  results
0  Pearson Chi-square ( 2.0) =   18.4783
1                    p-value =    0.0001
2                 Cramer's V =    0.1488 

Feature: Title 
                 Chi-square test  results
0  Pearson Chi-square ( 1.0) =    2.3626
1                    p-value =    0.1243
2               Cramer's phi =    0.0532 

Feature: Family 
                 Chi-square test  results
0  Pearson Chi-square ( 2.0) =   36.5618
1                    p-value =    0.0000
2                 Cramer's V =    0.2094 



In [45]:
# Visualizing correlations
corr_cat = pd.DataFrame(
    {'Feature' : ['Ticket Class', 'Sex', 'Port of Embarkation', 'Title', 'Family'],
     'Correlation' : [0.3063, 0.5253, 0.1505, 0.0526, 0.1651]
     }
).sort_values(by = 'Correlation')

corr_cat_plot = px.bar(
    corr_cat, 
    x = 'Feature', 
    y = 'Correlation', 
    color = 'Correlation',
    color_continuous_scale = px.colors.sequential.Teal,
    title = 'Correlations between Outcome and Categorical Features'
)

corr_cat_plot.update_layout(
    title_x = 0.5,
    width = 700,
    height = 500
)

corr_cat_plot.show(renderer = 'notebook_connected')

Correlations between categorical features and outcome feature are higher than those for numerical features. More specifically, four out of five features have a correlation with outcome feature higher than 0.1 as measured by Cramer's V or Cramer's Phi coefficient. Therefore, we can conclude that these features will be particularly useful for final predictions! Furthermore, obtained p-values for chi-square indicate that only 'Title' will not be useful for predictions! 

## 2.2. Feature Engineering

The goal of this section is to gather insights into interactions between different features when it comes to predicting passengers' survival. In other words, here we will ask questions such as *"Are male passengers more likely to survive if they were first-class passengers in comparison to second- and third-class passengers"*. These questions should indicate whethere there is a need for creating new features which will represent interaction terms. These interaction terms are often neglected, but they are a powerful tool for obtaining more accurate predictions.

This goal will be accomplished by asking questions, getting pivot tables, and visualizing data!

The correlation between *Sex* and *Surviving* is high (0.53), as well as the correlation between *Ticket Class* and *Surviving* (0.30). 
It is already known that women had advantage during the evacuation. However, it is reasonable to assume the advantage was not that emphasized for the first-class passengers (i.e. wealthy passengers, passengers of great importance, etc.). We can test whether men in first class were more likely to survive in comparison to men from second and third class. That should indicate the presence of interaction between *Sex* and *Ticket Class* which could be exploited for final predictions.

In [66]:
pd.pivot_table(titanic, values = 'Survived', index = 'Pclass', columns = 'Sex', aggfunc = 'mean')

Sex,Female,Male
Pclass,,
First Class,0.983333,0.368932
Second Class,0.921053,0.157407
Third Class,0.500000,0.137026


In [75]:
pd.pivot_table(titanic, values = 'Survived', index = 'Pclass', columns = 'Family')

Family,Alone,Big_f,Small_f
Pclass,,,
First Class,0.483871,0.800000,0.727273
Second Class,0.346154,0.717391,0.529412
Third Class,0.212963,0.283019,0.350877


In [74]:
pd.pivot_table(titanic, values = 'Survived', index = 'Sex', columns = 'Family')

Family,Alone,Big_f,Small_f
Sex,,,
Female,0.763158,0.595506,0.792208
Male,0.154791,0.282051,0.246377


In [84]:
from itertools import combinations

In [97]:
perm = combinations(['Pclass', 'Sex', 'Embarked', 'Title', 'Family'], 2)

In [98]:
perm = list(perm)

In [101]:
perm[0][0]

'Pclass'

# 3. Data Preprocessing

# 4. Model Development

# 5. Hyperparameter Tuning

# 6. Ensemble Models

# 7. Model Evaluation and Visualization

# 8. Concluding Remarks